In [3]:
!pip install transformers accelerate bitsandbytes langchain

     ---------------------------------------- 0.0/50.7 kB ? eta -:--:--
     ---------------------------------------- 50.7/50.7 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/810.5 kB ? eta -:--:--
   ----- ---------------------------------- 112.6/810.5 kB 3.3 MB/s eta 0:00:01
   ----------------------- ---------------- 481.3/810.5 kB 6.0 MB/s eta 0:00:01
   ---------------------------------------  809.0/810.5 kB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 810.5/810.5 kB 5.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------- ----------------- 1.0/1.8 MB 31.7 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 38.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/273.9 kB ? eta -:--:--
   ---------------------------------------- 273.9/273.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/86.9 kB ? eta -:--:--
   -----------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.


### Первым шагом попробуем инициализировать модель на устройстве и потестируем промпты

In [34]:
!transformers-cli env


Copy-and-paste the text below in your GitHub issue and FILL OUT the two last points.

- `transformers` version: 4.39.2
- Platform: Windows-10-10.0.22631-SP0
- Python version: 3.11.5
- Huggingface_hub version: 0.22.2
- Safetensors version: 0.4.2
- Accelerate version: 0.28.0
- Accelerate config: 	not found
- PyTorch version (GPU?): 2.2.0+cu121 (True)
- Tensorflow version (GPU?): not installed (NA)
- Flax version (CPU?/GPU?/TPU?): not installed (NA)
- Jax version: not installed
- JaxLib version: not installed
- Using GPU in script?: <fill in>
- Using distributed or parallel set-up in script?: <fill in>



### Загрузим модель на локальную машину

In [4]:
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

## Параметры кванизации

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_4bit = AutoModelForCausalLM.from_pretrained( model_id, device_map="auto",quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipeline)

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

C:\Users\croxw\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\croxw\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [8]:
device = "cuda"

In [28]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090'

#### Проверим как модель генерирует "из коробки"

In [18]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
#model.to(device)


txt = ["<s>[INST] Generate 5 different ads for keyword 'buy iphone 25' without emoji for 100 symbols. Put it on list [/INST]"]
generated_ids = llm.generate(txt, max_new_tokens=500, do_sample=True)
print(generated)

generations=[[Generation(text='<s>[INST] Generate 5 different ads for keyword \'buy iphone 25\' without emoji for 100 symbols. Put it on list [/INST] 1. "Buy an iPhone 25 for just $25 at our online store. Limited stock available. Hurry!"\n\n2. "Get the latest technology with an iPhone 25. Buy now for only $25. Free shipping!"\n\n3. "Own an iPhone 25 for an unbelievable price of $25. Secure your order today!"\n\n4. "Upgrade to a new iPhone 25. Buy it now for just $25 and enjoy a superior smartphone experience."\n\n5. "Bargain alert: iPhone 25 for only $25. Don\'t miss this amazing deal! Order now."\n\n- 1. "Buy an iPhone 25 with a discounted price of $25"\n- 2. "Get a brand new iPhone 25 for $25, limited time offer!"\n- 3. "Experience the power of an iPhone 25, now only $25"\n- 4. "Buy an iPhone 25 for the price of a cheap smartphone"\n- 5. "Score a deal with an iPhone 25, just $25!"\n\n[List]:\n1. Buy an iPhone 25 for just $25 at our online store.\n2. Get the latest technology with an 

##### На первый взгляд неплохо, но есть проблемы : модель допускает критичные неточности - в частности, описывает стоимость устройства, даёт информацию о скидках (

In [20]:
txt = ["<s>[INST] Generate 3 different ad title for keyword 'buy new ford mustang' without emoji for 40 symbols [/INST]"]
generated = llm.generate(txt, max_new_tokens=500, do_sample=True)
print(generated)

generations=[[Generation(text='<s>[INST] Generate 3 different ad title for keyword \'buy new ford mustang\' without emoji for 40 symbols [/INST] 1. "Buy a New Ford Mustang: Experience American Power"\n\n2. "New Ford Mustang for Sale: Modern Muscle and Thrills"\n\n3. "Join the Legend: Purchase a New Ford Mustang Today"\n\nAll of these titles are designed to appeal to potential buyers of a new Ford Mustang and convey a sense of excitement and desirability around the car. They also keep within the 40-symbol limit.')]] llm_output=None run=[RunInfo(run_id=UUID('eb8c0f0e-debc-4f91-8418-efc8c172d2ed'))]


In [55]:
for g in generated.generations[0][0]:
    print(g)

('text', '<s>[INST] Generate 3 different ad title for keyword \'buy new ford mustang\' without emoji for 40 symbols [/INST] 1. "Buy a New Ford Mustang: Experience American Power"\n\n2. "New Ford Mustang for Sale: Modern Muscle and Thrills"\n\n3. "Join the Legend: Purchase a New Ford Mustang Today"\n\nAll of these titles are designed to appeal to potential buyers of a new Ford Mustang and convey a sense of excitement and desirability around the car. They also keep within the 40-symbol limit.')
('generation_info', None)
('type', 'Generation')


In [47]:
dir(generated.dict)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [50]:
generated

LLMResult(generations=[[Generation(text='<s>[INST] Generate 3 different ad title for keyword \'buy new ford mustang\' without emoji for 40 symbols [/INST] 1. "Buy a New Ford Mustang: Experience American Power"\n\n2. "New Ford Mustang for Sale: Modern Muscle and Thrills"\n\n3. "Join the Legend: Purchase a New Ford Mustang Today"\n\nAll of these titles are designed to appeal to potential buyers of a new Ford Mustang and convey a sense of excitement and desirability around the car. They also keep within the 40-symbol limit.')]], llm_output=None, run=[RunInfo(run_id=UUID('eb8c0f0e-debc-4f91-8418-efc8c172d2ed'))])

In [56]:
txt = ["<s>[INST] Generate 3 different ads for keyword 'buy new ford mustang' without emoji for 100 symbols [/INST]"]
generated = llm.generate(txt, max_new_tokens=500, do_sample=True)
print(generated)

generations=[[Generation(text='<s>[INST] Generate 3 different ads for keyword \'buy new ford mustang\' without emoji for 100 symbols [/INST] 1. "Buy a New Ford Mustang and Experience Unmatched Power & Style. Limited Time Offer: Extra 5% Off MSRP."\n\n2. "Upgrade to a New Ford Mustang. Sleek Design, Powerful Engine, Advanced Technology. Financing Available."\n\n3. "Get the Heart-Pumping Performance of a New Ford Mustang. Order Yours Today & Save $1,000 on MSRP. Act Fast!"')]] llm_output=None run=[RunInfo(run_id=UUID('c1123e4d-65e2-48ba-acef-461f8de8a9ed'))]


In [57]:
txt = ["<s>[INST] There are five classes: 'auto','phone','animal','finance','travel'. What class has word 'Ford Mustang'? [/INST]"]
generated = llm.generate(txt, max_new_tokens=500, do_sample=True)
print(generated)

generations=[[Generation(text="<s>[INST] There are five classes: 'auto','phone','animal','finance','travel'. What class has word 'Ford Mustang'? [/INST] The class for the word 'Ford Mustang' would be 'auto' as it is a type of automobile.")]] llm_output=None run=[RunInfo(run_id=UUID('3f8313ef-dddc-41d8-a128-0744c44cdbf3'))]


In [60]:
model_4bit

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )